<a href="https://colab.research.google.com/github/anirutsolrak/chatBot-Layout/blob/main/C%C3%B3pia_de_Ol%C3%A1%2C_este_%C3%A9_o_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install dash-bootstrap-components
!pip install dash_table
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('chave')
genai.configure(api_key=GOOGLE_API_KEY)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.5 MB/s eta 0:00:00


In [30]:
from IPython.display import HTML, display
import google.generativeai as genai
import re
import json

model = genai.GenerativeModel('gemini-pro')

# Função para gerar as recomendações (sem alterações)
def gerar_recomendacoes(opcao, produto=None, objetivo=None, descricao=None, momento=None, busca=None, genero=None):
    if opcao == 'produtos':
        prompt = f"Recomendações de produtos:\nProduto: {produto}\nObjetivo: {objetivo}\nDescrição: {descricao}"
        response = model.generate_content(prompt)

        # Extrair informações da resposta usando expressões regulares
        produtos = []
        for match in re.finditer(r"Nome: (.+?)\nModelo: (.+?)\nPreço: (.+?)\nDescrição: (.+?)\nLink: (.+?)\n", response.text):
            produtos.append({
                'Nome': match.group(1),
                'Modelo': match.group(2),
                'Preço': match.group(3),
                'Descrição': match.group(4),
                'Link': match.group(5)
            })
        return produtos
    elif opcao == 'entretenimento':
        prompt = f"Recomendações de entretenimento:\nMomento: {momento}\nBusca: {busca}\nGênero: {genero}"
        response = model.generate_content(prompt)

        # Extrair informações da resposta usando expressões regulares
        entretenimento = []
        for match in re.finditer(r"Título: (.+?)\nGênero: (.+?)\nDescrição: (.+?)\nLink: (.+?)\n", response.text):
            entretenimento.append({
                'Título': match.group(1),
                'Gênero': match.group(2),
                'Descrição': match.group(3),
                'Link': match.group(4)
            })
        return entretenimento

# Função para exibir a interface HTML
def exibir_interface():
    html_string = """
    <!DOCTYPE html>
    <html>
    <head>
      <style>
        body {
          font-family: Arial, sans-serif;
        }
        h1 {
          color: blue;
          font-size: 24px;
        }
        table {
          border-collapse: collapse;
          width: 100%;
        }
        th, td {
          border: 1px solid black;
          padding: 8px;
          text-align: left;
        }
      </style>
    </head>
    <body>
      <h1>Chatbot de Recomendações</h1>
      <p>Escolha uma opção:</p>
      <select id="opcao">
        <option value="produtos">Produtos</option>
        <option value="entretenimento">Entretenimento</option>
      </select>
      <br><br>
      <div id="form-produtos">
        <label for="produto">Qual produto você está procurando?</label><br>
        <input type="text" id="produto" name="produto"><br><br>
        <label for="objetivo">Qual o seu objetivo com a compra?</label><br>
        <input type="text" id="objetivo" name="objetivo"><br><br>
        <label for="descricao">Descreva o produto ideal para você:</label><br>
        <input type="text" id="descricao" name="descricao"><br><br>
      </div>
      <div id="form-entretenimento" style="display: none;">
        <label for="momento">Qual o seu momento atual?</label><br>
        <input type="text" id="momento" name="momento"><br><br>
        <label for="busca">O que você está buscando?</label><br>
        <input type="text" id="busca" name="busca"><br><br>
        <label for="genero">Quais são seus gêneros favoritos?</label><br>
        <input type="text" id="genero" name="genero"><br><br>
      </div>
      <button onclick="gerarRecomendacoes()">Gerar Recomendações</button>
      <br><br>
      <div id="resultados"></div>

      <script>
        function exibirFormulario(opcao) {
          if (opcao === 'produtos') {
            document.getElementById('form-produtos').style.display = 'block';
            document.getElementById('form-entretenimento').style.display = 'none';
          } else {
            document.getElementById('form-produtos').style.display = 'none';
            document.getElementById('form-entretenimento').style.display = 'block';
          }
        }

        async function gerarRecomendacoes() {
          var opcao = document.getElementById('opcao').value;
          var produto = document.getElementById('produto').value;
          var objetivo = document.getElementById('objetivo').value;
          var descricao = document.getElementById('descricao').value;
          var momento = document.getElementById('momento').value;
          var busca = document.getElementById('busca').value;
          var genero = document.getElementById('genero').value;

          const result = await google.colab.kernel.invokeFunction(
            'gerar_recomendacoes',
            [opcao, produto, objetivo, descricao, momento, busca, genero],
            {}
          );
          const data = result.data['application/json'];
          exibirResultados(data);
        }

        function exibirResultados(data) {
          var resultadosDiv = document.getElementById('resultados');
          resultadosDiv.innerHTML = ''; // Limpa os resultados anteriores

          if (data.length === 0) {
            resultadosDiv.innerHTML = '<p>Nenhuma recomendação encontrada.</p>';
            return;
          }

          var tabela = document.createElement('table');
          var thead = document.createElement('thead');
          var tbody = document.createElement('tbody');

          if (data[0].hasOwnProperty('Nome')) { // Verifica se é uma recomendação de produto
            var headerRow = document.createElement('tr');
            ['Nome', 'Modelo', 'Preço', 'Descrição', 'Link'].forEach(function(coluna) {
              var th = document.createElement('th');
              th.textContent = coluna;
              headerRow.appendChild(th);
            });
            thead.appendChild(headerRow);

            data.forEach(function(produto) {
              var row = document.createElement('tr');
              ['Nome', 'Modelo', 'Preço', 'Descrição', 'Link'].forEach(function(coluna) {
                var td = document.createElement('td');
                td.textContent = produto[coluna];
                row.appendChild(td);
              });
              tbody.appendChild(row);
            });
          } else if (data[0].hasOwnProperty('Título')) { // Verifica se é uma recomendação de entretenimento
            var headerRow = document.createElement('tr');
            ['Título', 'Gênero', 'Descrição', 'Link'].forEach(function(coluna) {
              var th = document.createElement('th');
              th.textContent = coluna;
              headerRow.appendChild(th);
            });
            thead.appendChild(headerRow);

            data.forEach(function(item) {
              var row = document.createElement('tr');
              ['Título', 'Gênero', 'Descrição', 'Link'].forEach(function(coluna) {
                var td = document.createElement('td');
                td.textContent = item[coluna];
                row.appendChild(td);
              });
              tbody.appendChild(row);
            });
          }

          tabela.appendChild(thead);
          tabela.appendChild(tbody);
          resultadosDiv.appendChild(tabela);
        }

        // Chamar a função exibirFormulario ao carregar a página
        window.onload = function() {
          exibirFormulario(document.getElementById('opcao').value);
        };

        // Adicionar um evento de mudança ao select
        document.getElementById('opcao').addEventListener('change', function() {
          exibirFormulario(this.value);
        });
      </script>
    </body>
    </html>
    """
    display(HTML(html_string))

# Chamar a função para exibir a interface
exibir_interface()